In [17]:
#AABB VERSION

import cv2

# Path to your image and annotation file
image_path = "/Users/palbentsen/Desktop/master/Train/60/images/60_frame_500.jpg"
annotation_path = "/Users/palbentsen/Desktop/master/Train/60/labels/60_frame_500.txt"

# Load the image
image = cv2.imread(image_path)

# Image dimensions
height, width = image.shape[:2]

# Function to convert normalized coordinates to pixel coordinates
def convert(x, y, w, h, img_width, img_height):
    x_pixel = int((x - w / 2) * img_width)
    y_pixel = int((y - h / 2) * img_height)
    w_pixel = int(w * img_width)
    h_pixel = int(h * img_height)
    return x_pixel, y_pixel, w_pixel, h_pixel

# Read the annotations and draw rectangles on the image
with open(annotation_path, 'r') as file:
    for line in file.readlines():
        _, x, y, w, h = map(float, line.split())
        x_pixel, y_pixel, w_pixel, h_pixel = convert(x, y, w, h, width, height)
        cv2.rectangle(image, (x_pixel, y_pixel), (x_pixel + w_pixel, y_pixel + h_pixel), (0, 255, 0), 2)

# Define whether to show the image or save it
showImageMode = False  # Change this to False to save the image instead of displaying it

if showImageMode:
    # Display the image with annotations
    cv2.imshow("Annotated Image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    # Save the annotated image to disk
    save_path = "/Users/palbentsen/Desktop/master/outputImages1march/annotated_image.jpg"  # Specify your save path here
    cv2.imwrite(save_path, image)
    print(f"Annotated image has been saved to: {save_path}")


Annotated image has been saved to: /Users/palbentsen/Desktop/master/outputImages1march/annotated_image.jpg


In [20]:
#OBB VERSION

import cv2
import numpy as np

# Path to your image and OBB annotation file
image_path = "/Users/palbentsen/Desktop/master/outputImages1march/p60/p60-f500-GT+silverAABB.jpg"
annotation_path = "/Users/palbentsen/Desktop/silver-obbTemp/60_frame_500.txt"  # Update this path

# Load the image
image = cv2.imread(image_path)

# Image dimensions
height, width = image.shape[:2]

# Function to convert normalized coordinates to pixel coordinates
def convert_obb(coordinates, img_width, img_height):
    pixel_coordinates = [(int(x * img_width), int(y * img_height)) for x, y in zip(coordinates[::2], coordinates[1::2])]
    return np.array(pixel_coordinates, np.int32)

# Read the annotations and draw polygons on the image
with open(annotation_path, 'r') as file:
    for line in file.readlines():
        parts = line.split()
        class_id = int(parts[0])  # Not used here, but you might want to use it for color coding etc.
        coordinates = list(map(float, parts[1:]))
        polygon = convert_obb(coordinates, width, height)
        cv2.polylines(image, [polygon], isClosed=True, color=(0, 0, 255), thickness=2)

# Define whether to show the image or save it
showImageMode = False  # Change this to False to save the image instead of displaying it

if showImageMode:
    # Display the image with annotations
    cv2.imshow("Annotated Image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    # Save the annotated image to disk
    save_path = "/Users/palbentsen/Desktop/master/outputImages1march/p60/p60-f500-GT+silverAABB+OBB.jpg"  # Specify your save path here
    cv2.imwrite(save_path, image)
    print(f"Annotated image has been saved to: {save_path}")


Annotated image has been saved to: /Users/palbentsen/Desktop/master/outputImages1march/p60/p60-f500-GT+silverAABB+OBB.jpg


In [2]:
import cv2
import numpy as np

# Input paths
#image_path = "/Users/palbentsen/Desktop/master/Train/54/images/54_frame_500.jpg"
#image_path = "/Users/palbentsen/Desktop/master/obb_Inference/runs/detect/cleanedRTDETR-54-f500/54_frame_500.jpg"
image_path = "/Users/palbentsen/Desktop/master/obb_Inference/runs/detect/legacyRTDETRnonSSL-54-f500/54_frame_500.jpg"
aabb_label_path = "/Users/palbentsen/Desktop/master/Train/54/labels/54_frame_500.txt"  # Update this path
aabb_label_path_2 = "/Users/palbentsen/Desktop/master/Train/54/labels/54_frame_500.txt"  # Update this path
#aabb_label_path_2 = "/Users/palbentsen/Desktop/silver-aabbTemp/54_frame_500.txt"  # Update this path
obb_label_path = "/Users/palbentsen/Desktop/silver-obbTemp/54_frame_500.txt"  # Update this path
obb_label_path_2 = "/Users/palbentsen/Desktop/master/obb_Inference/runs/obb/obbYOLOv8s-54-f500/labels/54_frame_500.txt"  # Update this path

# Load the image
image = cv2.imread(image_path)

# Image dimensions
height, width = image.shape[:2]

# Convert normalized coordinates to pixel coordinates
def convert(x, y, w, h, img_width, img_height):
    x_pixel = int((x - w / 2) * img_width)
    y_pixel = int((y - h / 2) * img_height)
    w_pixel = int(w * img_width)
    h_pixel = int(h * img_height)
    return x_pixel, y_pixel, w_pixel, h_pixel

def convert_obb(coordinates, img_width, img_height):
    return np.array([(int(x * img_width), int(y * img_height)) for x, y in zip(coordinates[::2], coordinates[1::2])], np.int32)

# Function to draw labels
def draw_labels(label_path, color, convert_func, is_obb=False):
    with open(label_path, 'r') as file:
        for line in file.readlines():
            parts = line.split()
            if not is_obb:
                _, x, y, w, h = map(float, parts)
                x_pixel, y_pixel, w_pixel, h_pixel = convert_func(x, y, w, h, width, height)
                cv2.rectangle(image, (x_pixel, y_pixel), (x_pixel + w_pixel, y_pixel + h_pixel), color, 1)
            else:
                coordinates = list(map(float, parts[1:]))
                polygon = convert_func(coordinates, width, height)
                cv2.polylines(image, [polygon], isClosed=True, color=color, thickness=1, lineType=cv2.LINE_AA)

# Boolean flags to control drawing
DrawAABBlabel1 = False
DrawAABBlabel2 = True
DrawOBBlabel1 = False
DrawOBBlabel2 = False

# Draw labels based on flags
if DrawAABBlabel1:
    draw_labels(aabb_label_path, (0, 0, 255), convert)  # red?
if DrawAABBlabel2:
    draw_labels(aabb_label_path_2, (255, 255, 255), convert)  # white
if DrawOBBlabel1:
    draw_labels(obb_label_path, (255, 255, 255), convert_obb, is_obb=True)  # Red
if DrawOBBlabel2:
    draw_labels(obb_label_path_2, (255, 0, 255), convert_obb, is_obb=True)  # Purple

# Save or display the image
cv2.imshow("Annotated Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [52]:
#SEG VERSION
import cv2
import numpy as np
import random

def draw_segmentation_masks(image_path, mask_txt_path):
    # Load the image
    image = cv2.imread(image_path)
    height, width = image.shape[:2]

    # Define colors for each class (you can expand this list for more classes)
    colors = {
        0: (0, 255, 0),  # Green for class 0
        # Add more colors for more classes if necessary
    }

    # Read the mask file
    with open(mask_txt_path, 'r') as f:
        for line in f:
            parts = line.split()
            class_id = int(parts[0])
            # Convert normalized coordinates to pixel coordinates
            points = np.array([[int(float(x) * width), int(float(y) * height)] for x, y in zip(parts[1::2], parts[2::2])], np.int32)
            points = points.reshape((-1, 1, 2))
            # Draw the polygon
            cv2.polylines(image, [points], isClosed=True, color=colors[class_id], thickness=2)

    # Display the result
    cv2.imshow('Segmentation Masks', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def draw_filled_segmentation_masks(image_path, mask_txt_path):
    # Load the image
    image = cv2.imread(image_path)
    height, width = image.shape[:2]

    # Create an empty mask to draw all segmentation masks
    mask_image = np.zeros_like(image)

    # Define colors for each class (you can expand this list for more classes)
    colors = {
        0: (0, 255, 0),  # Green for class 0
        # Add more colors for more classes if necessary
    }

    # Read the mask file
    with open(mask_txt_path, 'r') as f:
        for line in f:
            parts = line.split()
            class_id = int(parts[0])
            # Convert normalized coordinates to pixel coordinates
            points = np.array([[int(float(x) * width), int(float(y) * height)] for x, y in zip(parts[1::2], parts[2::2])], np.int32)
            points = points.reshape((-1, 1, 2))
            # Draw the filled polygon on the mask image
            cv2.fillPoly(mask_image, [points], color=colors[class_id])

    # Blend the mask image with the original image
    # Adjust alpha (transparency factor) as needed
    alpha = 0.4
    overlayed_image = cv2.addWeighted(mask_image, alpha, image, 1 - alpha, 0)

    # Display the result
    cv2.imshow('Filled Segmentation Masks', overlayed_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
""" 
def draw_filled_segmentation_masks_with_random_colors(image_path, mask_txt_path, save_path=None, showImageMode=True):
    # Load the image
    image = cv2.imread(image_path)
    height, width = image.shape[:2]

    # Create an empty mask to draw all segmentation masks
    mask_image = np.zeros_like(image)

    # Function to generate a random color
    def random_color():
        return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

    # Read the mask file
    with open(mask_txt_path, 'r') as f:
        for line in f:
            parts = line.split()
            # Generate a random color for this instance
            color = random_color()
            # Convert normalized coordinates to pixel coordinates
            points = np.array([[int(float(x) * width), int(float(y) * height)] for x, y in zip(parts[1::2], parts[2::2])], np.int32)
            points = points.reshape((-1, 1, 2))
            # Draw the filled polygon on the mask image with the random color
            cv2.fillPoly(mask_image, [points], color=color)

    # Blend the mask image with the original image
    # Adjust alpha (transparency factor) as needed
    alpha = 0.3
    overlayed_image = cv2.addWeighted(mask_image, alpha, image, 1 - alpha, 0)

    if showImageMode:
        # Display the result
        cv2.imshow('Filled Segmentation Masks with Random Colors', overlayed_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        # Save the result to the specified path
        if save_path is not None:
            cv2.imwrite(save_path, overlayed_image)
            print(f"Annotated image has been saved to: {save_path}")
        else:
            print("Save path is not specified.")
 """

def draw_filled_segmentation_masks_with_random_colors(image_path, mask_txt_path, save_path=None, showImageMode=True):
    # Load the image
    image = cv2.imread(image_path)
    height, width = image.shape[:2]

    # Create an empty image for drawing masks (filled with zeros)
    mask_overlay = np.zeros_like(image)

    # Function to generate a random color
    def random_color():
        if randomEnabled:
            return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        else: 
            return (0, 255, 255) #White

    # Read the mask file
    with open(mask_txt_path, 'r') as f:
        for line in f:
            parts = line.split()
            # Generate a random color for this instance
            color = random_color()
            # Convert normalized coordinates to pixel coordinates
            points = np.array([[int(float(x) * width), int(float(y) * height)] for x, y in zip(parts[1::2], parts[2::2])], np.int32)
            points = points.reshape((-1, 1, 2))
            # Draw the filled polygon on the mask_overlay image with the random color
            cv2.fillPoly(mask_overlay, [points], color=color)

    # Now blend only the non-zero (mask) areas of mask_overlay with the original image
    mask_indices = np.where(mask_overlay != [0, 0, 0])
    image[mask_indices] = cv2.addWeighted(image, 1 - alpha, mask_overlay, alpha, 0)[mask_indices]

    if showImageMode:
        # Display the result
        cv2.imshow('Filled Segmentation Masks with Random Colors', image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        # Save the result to the specified path
        if save_path is not None:
            cv2.imwrite(save_path, image)
            print(f"Annotated image has been saved to: {save_path}")
        else:
            print("Save path is not specified.")

# Adjust the alpha value as needed
alpha = 0.55  # Example alpha value
randomEnabled = False  # Set to True to enable random colors for each instance

# Example usage
image_path = "/Users/palbentsen/Desktop/master/Train/60/images/60_frame_500.jpg"
#image_path = '/Users/palbentsen/Desktop/master/outputImagesDrawFunction/p60/p60-f500.jpg'
mask_txt_path = '/Users/palbentsen/Desktop/output-SAM-size-B/60_frame_500.txt'
save_path = '/Users/palbentsen/Desktop/master/outputImagesDrawFunction/p60/p60-MASK-middleAlpha-sizeB.jpg'
showImageMode = False  # Change to True if you want to display the image instead of saving
draw_filled_segmentation_masks_with_random_colors(image_path, mask_txt_path, save_path, showImageMode)


Annotated image has been saved to: /Users/palbentsen/Desktop/master/outputImagesDrawFunction/p60/p60-MASK-middleAlpha-sizeB.jpg
